In [ ]:

!pip install rdflib

import urllib.request
url = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/refs/heads/master/Assignment4/course_materials/python/validation.py"
urllib.request.urlretrieve(url, "validation.py")

from validation import Report

# Carga del grafo RDF
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS

g = Graph()
g.namespace_manager.bind("ns", Namespace("http://somewhere#"), override=False)

github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/master/Assignment4/course_materials"
g.parse(github_storage + "/rdf/data06.ttl", format="turtle")

report = Report()


# TASK 7.1a — RDFLib: listar (clase, superclase)

result = []

# Recorremos todas las clases definidas en el grafo
for cls in set(g.subjects(RDF.type, RDFS.Class)):
    
    # Intentamos obtener la superclase (si no existe devuelve None)
    superclass = g.value(subject=cls, predicate=RDFS.subClassOf, object=None)
    
    # Guardamos la tupla (clase, superclase)
    result.append((cls, superclass))

# Visualización
print("\n--- TASK 7.1a: (clase, superclase) RDFLib ---")
for c, s in result:
    print(c, " -> ", s)

# Validación
report.validate_07_1a(result)



# TASK 7.1b — SPARQL: mismas tuplas ?c ?sc

query_7_1b = """
    SELECT DISTINCT ?c ?sc WHERE {
        ?c a rdfs:Class .
        OPTIONAL { ?c rdfs:subClassOf ?sc . }
    }
"""

print("\n--- TASK 7.1b: SPARQL resultados ---")
for row in g.query(query_7_1b):
    print(row.c, " -> ", row.sc)

report.validate_07_1b(query_7_1b, g)



# TASK 7.2a — RDFLib: individuos de Person (incluye subclases)

ns_people = Namespace("http://oeg.fi.upm.es/def/people#")

# Conjunto de clases relevantes (Person + subclases transitivas)
relevant_classes = {ns_people.Person} | set(
    g.transitive_subjects(RDFS.subClassOf, ns_people.Person)
)

# Obtenemos todos los individuos cuyo rdf:type sea alguna de esas clases
individuals = sorted(
    {ind for c in relevant_classes for ind in g.subjects(RDF.type, c)},
    key=str
)

print("\n--- TASK 7.2a: Individuos de Person ---")
for ind in individuals:
    print(ind)

report.validate_07_02a(individuals)



# TASK 7.2b — SPARQL: mismos individuos ?ind

from rdflib.plugins.sparql import prepareQuery

query_7_2b = prepareQuery("""
    SELECT ?ind WHERE {
        ?ind rdf:type ?c .
        ?c rdfs:subClassOf* ns:Person .
    }
""", initNs={"ns": ns_people, "rdf": RDF, "rdfs": RDFS})

print("\n--- TASK 7.2b: SPARQL Individuos ---")
for row in g.query(query_7_2b):
    print(row.ind)

report.validate_07_02b(g, query_7_2b)



# TASK 7.3 — nombre + tipo de quienes conocen a Rocky

query_7_3 = prepareQuery("""
    SELECT ?name ?type WHERE {
        ?x ns:knows ns:Rocky .
        ?x rdf:type ?type .
        ?x rdfs:label ?name .
    }
""", initNs={"ns": ns_people, "rdf": RDF, "rdfs": RDFS})

print("\n--- TASK 7.3: Conocen a Rocky ---")
for row in g.query(query_7_3):
    print(row.name, " - ", row.type)

report.validate_07_03(g, query_7_3)



# TASK 7.4 — nombre de quienes tienen colega o colega-de-colega

query_7_4 = prepareQuery("""
    SELECT DISTINCT ?name WHERE {
        { 
          ?x rdfs:label ?name .
          ?x ns:hasColleague ?y .
        }
        UNION
        {
          ?x rdfs:label ?name .
          ?x ns:hasColleague ?y .
          ?y ns:hasColleague ?z .
        }
    }
""", initNs={"ns": ns_people, "rdfs": RDFS})

print("\n--- TASK 7.4: Colegas y colegas-de-colegas ---")
for row in g.query(query_7_4):
    print(row.name)

report.validate_07_04(g, query_7_4)



# GUARDAR REPORTE FINAL

report.save_report("_Task_07")

